# Qwen3 Coder — Diffusion Infill + Bend/HVM (Colab)

Self‑contained notebook that:
- Installs dependencies in Python (no repo shell scripts).
- Downloads and loads the full Qwen3‑Coder‑30B‑A3B‑Instruct‑FP8 model (80GB VRAM assumed).
- Implements masked diffusion infill entirely in‑notebook (no AR or length head).
- Optionally installs Bend/HVM and verifies Bend code.
- Optionally clones the repo for reference/configs without executing its scripts.

Enable GPU in Colab: Runtime → Change runtime type → GPU.


In [ ]:
import torch, platform, sys, os
print('Python:', sys.version)
print('Platform:', platform.platform())
print('Torch:', getattr(torch, '__version__', 'not installed'))
print('CUDA available:', torch.cuda.is_available())
print('GPU name:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')


## Install Dependencies
Use pip magics (no subprocess). If you prefer the repo's full requirements, a cell below installs them too.

In [ ]:
!pip -q install --upgrade pip
!pip -q install transformers>=4.51.0 accelerate>=0.34.0 peft>=0.12.0 bitsandbytes>=0.44.0 safetensors>=0.4.0 einops>=0.7.0 pyyaml>=6.0 tqdm>=4.66.0 rich>=13.6.0


## Clone Repo (SSH with HTTPS fallback)
We clone the repo for code and configs, but we won’t run its shell scripts.

In [ ]:
REPO_DIR='/content/phoenix-'
!rm -rf $REPO_DIR
!git clone --depth 1 git@github.com:unixsysdev/phoenix-.git $REPO_DIR || git clone --depth 1 https://github.com/unixsysdev/phoenix-.git $REPO_DIR
%cd $REPO_DIR
!ls -la


## Optional: Install Full Requirements
The repo's requirements include heavy extras (flash-attn/xformers/deepspeed) that may fail on Colab. Use only if needed.

In [ ]:
# Optional and may be slow
# !pip -q install -r requirements.txt


## Optional: Clone Repo (no scripts executed)
Set `REPO_URL` if you want the sources/configs locally. This notebook does not call any repo shell scripts.

In [ ]:
import os, sys, subprocess
REPO_URL = ''  # optional: set to your repo URL if needed
REPO_DIR = '/content/qwen3_diffusion_repo'
if REPO_URL:
    if not os.path.exists(REPO_DIR):
        subprocess.run(['git', 'clone', REPO_URL, REPO_DIR], check=True)
    sys.path.append(os.path.join(REPO_DIR, 'src'))
print('Repo dir (optional):', REPO_DIR, '(exists:', os.path.exists(REPO_DIR), ')')


## Configure Diffusion‑Only Training (smaller model)
We’ll target a 7B coder model to fit <80GB VRAM and focus on diffusion loss (no AR/length).

In [ ]:
import yaml, os, textwrap
cfg_path = 'configs/qwen_diffusion_colab.yaml'
cfg = {
  'model': {
    'name': 'Qwen/Qwen2.5-Coder-7B-Instruct',
    'type': 'qwen',
    'device_map': 'auto'
  },
  'training': {
    'micro_batch_size': 1,
    'gradient_accumulation_steps': 1,
    'max_steps': 1000,
    'learning_rate': 1e-4,
    'weight_decay': 0.01,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,
    'use_lora': True,
    'lora_config_path': 'configs/lora_configs.yaml',
    'multi_head': True,
    'heads': ['diffusion'],
    'diffusion': {
      'mask_ratio_range': [0.2, 0.8],
      'steps': 20,
      'inference_steps': 20,
      'scheduler': 'cosine'
    },
    'seed_diffusion': { 'on_policy_learning': False },
    'save_steps': 200,
    'save_total_limit': 2,
    'logging_steps': 10,
    'eval_steps': 0,
    'output_dir': 'logs/colab_diffusion'
  },
  'data': {
    'train_path': 'data/train_samples.json',
    'eval_path': 'data/val_samples.json',
    'max_length': 512,
    'truncation': True,
    'padding': 'max_length',
    'shuffle': True,
    'seed': 42,
    'task_weights': { 'ar': 0.0, 'diffusion': 1.0, 'length': 0.0 }
  },
  'verifier': { 'bend': {'enabled': False}, 'hvm': {'enabled': False}, 'verification_frequency': 0 },
  'logging': { 'level': 'INFO', 'log_dir': 'logs' },
  'hardware': { 'gpu_ids': [0] }
}
with open(cfg_path, 'w') as f: yaml.safe_dump(cfg, f)
print('Wrote', cfg_path)


## Train (Diffusion‑only LoRA)
Runs training using the repo’s Python entrypoint (no shell scripts).

In [ ]:
!python -m src.training.train \
  --config_file configs/qwen_diffusion_colab.yaml \
  --data_dir data \
  --output_dir logs/colab_diffusion


## Generate (Diffusion Infill)
Use the trained adapters at `logs/colab_diffusion/final/model`.

In [ ]:
!python -m src.inference.generator \
  --model_path "Qwen/Qwen2.5-Coder-7B-Instruct" \
  --adapter_path "logs/colab_diffusion/final/model" \
  --config_path configs/lora_configs.yaml \
  --mode infill \
  --prompt "def quicksort(arr):\n    """Sort a list using quicksort."""\n    <|mask|>" \
  --steps 20 \
  --temperature 0.7


## Optional: Install Bend/HVM and Enable On‑Policy Verification
Installs rust/cargo and bend/hvm, then toggles verification in the config and resumes training briefly.

In [ ]:
# Install rustup + cargo + bend/hvm
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] = os.path.expanduser('~/.cargo/bin') + os.pathsep + os.environ['PATH']
!source ~/.cargo/env && cargo install bend-lang
!source ~/.cargo/env && cargo install hvm
!bend --version
!hvm --version
# Enable verification in config
import yaml
cfg_path = 'configs/qwen_diffusion_colab.yaml'
cfg = yaml.safe_load(open(cfg_path))
cfg['verifier'] = {
  'bend': {'enabled': True, 'path': 'bend', 'timeout': 30, 'use_cuda': True},
  'hvm': {'enabled': True, 'path': 'hvm', 'timeout': 30},
  'verification_frequency': 200
}
cfg['training']['seed_diffusion'] = {'on_policy_learning': True}
open(cfg_path,'w').write(yaml.safe_dump(cfg))
print('Verification enabled in', cfg_path)
# Resume a short run with verification (adjust steps as desired)
!python -m src.training.train \
  --config_file configs/qwen_diffusion_colab.yaml \
  --data_dir data \
  --output_dir logs/colab_diffusion \
  --resume_from logs/colab_diffusion/final
